# Getting Started
Using Conversational React Agent to interact with our llm tools

In [ ]:
from togetherllm import TogetherLLM
from langchain.tools import DuckDuckGoSearchRun

from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.agents import load_tools
from langchain.agents import Tool, AgentExecutor, initialize_agent

from langchain.agents import AgentType

from pprint import pprint

from langchain.chains.summarize import load_summarize_chain
from langchain.chains import LLMMathChain

from langchain.embeddings import HuggingFaceEmbeddings
from langchain import PromptTemplate, LLMChain

from langchain.utilities import SerpAPIWrapper
import os


In [ ]:
llm = TogetherLLM(
    model= "togethercomputer/llama-2-7b-chat",
    temperature=0,
    max_tokens=512
)

## Let's create our Tool
- 1st Tool : To create summarizer tools
- 2nd Tool : To create Q&A tools
- 3rd Tool : To create LLM Math Chain tools
- 4th Tool : To create a general tools

In [386]:
# Here we combine the tools into in one single dictionary

### Summarize
summarize_chain = load_summarize_chain(llm, chain_type="map_reduce")

### QA closed book
custom_prompt_template = """[INST] <<SYS>>
Your name is Kelly, You are helpful assistant, you always open and only answer for the assistant then you stop. 
If you don't know the answer, just say you don't know and submit the request to hotline@xfab.com for further assistance.
<</SYS>>

Question: {question}
[/INST]"""

#prompt = PromptTemplate(template="Question: {question}\nAnswer:", input_variables=["question"])

prompt = PromptTemplate(input_variables=['question'],template=custom_prompt_template)
qachain = LLMChain(llm=llm, prompt=prompt)

### Math 
llm_math = LLMMathChain(llm=llm, verbose=True)

## Duckduckgosearch
search = DuckDuckGoSearchRun()

## Serapi
os.environ["SERPAPI_API_KEY"] = "f078f92ace79931f581dcb74e38817617d839c30aed3bd08c834ac46c1942ff2"
serp = SerpAPIWrapper()

tools = [
    # Tool(
    #     name = 'Summarization',
    #     func=summarize_chain.run,
    #     description="Use this tool when summarizing information."
    # ),
    
    # Tool(
    #     name="Calculator",
    #     func=llm_math.run,
    #     description="Use this tool when you need to answer questions about mathematics"
    # ),

    Tool(
        name="Question Answer",
        func=qachain.run,
        description="Use this tool for question and answer. Use this more than the normal search if the question is about general knowledge and programming, like 'can you write me a letter?' or 'write me a song or poem?' or 'write me a script or code?' "
    ),

    # Tool(
    #     name="DuckDuckGo Search",
    #     func=search.run,
    #     description="Use this tool when you need to do a search on the internet to find information that another tool can't find. Be specific with your input."
    # ),

    Tool(
        name="SERPAPI Search",
        func=serp.run,
        description="Useful for when you need to answer questions about current events.Use this more than the normal search if the question is about time and date, like 'what time is it now?' or 'what time is it now at Kuching?"
    )
]

## Here we'll initialize the agent 

In [387]:
# Using conversational react with memory
memory = ConversationBufferMemory(memory_key="chat_history")
agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=4,
    memory=memory,
    handle_parsing_errors="Check your output and make sure it conforms!",
    early_stopping_method="generate", #to perform one FINAL pass through the LLM to generate an output
)

In [ ]:
agent.run("Write me a python code that read ultrasonic sensor and trigger the buzzer if distance is less than 5cm?")

In [ ]:
try:
    response= agent.run("Write me a python code that read ultrasonic sensor and trigger the buzzer if distance is less than 5cm?")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [ ]:
# question = "What time is it now at kuching in Local time ?"
# inputs = "https://www.google.com/search?q=" + question.replace(" ", "+"),

try:
    response= agent.run("Write me a resignation letter. Please ensure the letter consist my name Lukas and my position as software engineer.")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")

In [392]:
try:
    response= agent.run("What time is it now at Kuching ?")
    print(response)
except Exception as e:
        response = str(e)
        if response.startswith("Could not parse LLM output: `"):
            response = response.removeprefix("Could not parse LLM output: `").removesuffix("`")



> Entering new AgentExecutor chain...
Could not parse LLM output: `
Please provide your response.`
Observation: Check your output and make sure it conforms!
Thought:Do I need to use a tool? No
Action: question answer
Action Input: what time is it now at Kuching ?

Please provide your response.
Observation: question answer is not a valid tool, try one of [Question Answer, SERPAPI Search].
Thought:Do I need to use a tool? Yes
Action: question answer
Action Input: what time is it now at Kuching ?

Please provide your response.
Observation: I'm not sure what you're asking. Can you please clarify?
Thought:Do I need to use a tool? Yes
Action: question answer
Action Input: what time is it now at Kuching ?

Please provide your response.
Observation: I'm not sure what you're asking. Can you please clarify?
Thought:Do I need to use a tool? Yes
Action: question answer
Action Input: what time is it now at Kuching ?

Please provide your response.
Observation: I'm not sure what you're asking. Can 